# Fires



In [1]:
from bsk_rl import sats, act, obs
from bsk_rl.sim import dyn, fsw
import numpy as np

class FireSat(sats.FireSatellite):
    observation_spec = [
        obs.OpportunityProperties(
            dict(prop="opportunity_open", norm=5700.0),
            n_ahead_observe=10,
            type="fire",
        )
    ]
    action_spec = [act.Image(n_ahead_image=10)]
    dyn_type = dyn.FullFeaturedDynModel
    fsw_type = fsw.SteeringImagerFSWModel

In [2]:

from bsk_rl.utils.orbital import random_orbit

STEERING_SAT_ARGS = dict(
    imageAttErrorRequirement=0.01,
    imageRateErrorRequirement=0.01,
    batteryStorageCapacity=80.0 * 3600 * 100,
    storedCharge_Init=80.0 * 3600 * 100.0,
    dataStorageCapacity=200 * 8e6 * 100,
    u_max=0.4,
    imageTargetMinimumElevation=np.arctan(800 / 500),
    # wheelSpeeds=np.ones(3) * 0,
    K1=0.25,
    K3=3.0,
    omega_max=np.radians(5),
    servo_Ki=5.0,
    servo_P=150 / 5,
    oe=lambda: random_orbit(alt=800, i=55),
)


In [3]:
from bsk_rl import SatelliteTasking
from bsk_rl.scene.fires.fires import Fires
from bsk_rl.data.fire_data import FireReward

env = SatelliteTasking(
    satellite=FireSat("FireSat-1", STEERING_SAT_ARGS, scan_ahead_time=420.0),
    scenario=Fires(horizon=5700*15),
    rewarder=FireReward(reimage_post_burn=True),
    time_limit=5700*15,
    log_level="INFO",
    max_step_duration=5*60,
)
times = []
soons = []
env.reset()
truncated = False
while not truncated:
    observation, reward, terminated, truncated, info = env.step(env.action_space.sample())

    soon = [op for op, (start, close) in env.unwrapped.satellite.next_opportunities_dict().items() if start < env.unwrapped.simulator.sim_time + env.unwrapped.satellite.scan_ahead_time]
    # env.unwrapped.satellite.logger.info(f"Soon {len(soon)}")
    soons.append(len(soon))
    times.append(env.unwrapped.simulator.sim_time)
print(times)
print(soons)

2024-07-02 18:16:20,970 gym                            INFO       Resetting environment with seed=156270912
2024-07-02 18:16:22,233 data.fire_data                 INFO       Initial fires: 1693
2024-07-02 18:16:22,327 sats.satellite.FireSat-1       INFO       <0.00> FireSat-1: Finding opportunity windows from 0.00 to 420.00 seconds
2024-07-02 18:16:22,485 data.fire_data                 INFO       <0.00> Found fire Fire(-16.23,48.87) (burn interval: -3785.6 - 55066.1)
2024-07-02 18:16:22,485 sats.satellite.FireSat-1       INFO       <0.00> FireSat-1: Finding opportunity windows from 600.00 to 86100.00 seconds
2024-07-02 18:16:26,289 gym                            INFO       <0.00> Satellites requiring retasking: ['FireSat-1_11983794960']
2024-07-02 18:16:26,290 gym                            INFO       <0.00> Environment reset
2024-07-02 18:16:26,291 gym                            INFO       <0.00> === STARTING STEP ===
2024-07-02 18:16:26,291 sats.satellite.FireSat-1       INFO       <

[300.0, 350.0, 357.0, 385.0, 406.0, 706.0, 1006.0000000000001, 1306.0, 1606.0, 1906.0000000000002, 2206.0, 2506.0, 2806.0, 3106.0, 3406.0, 3706.0000000000005, 3913.0000000000005, 3957.0000000000005, 3990.0000000000005, 4165.0, 4465.0, 4765.0, 5065.0, 5365.0, 5665.0, 5965.0, 6011.0, 6053.0, 6098.0, 6142.0, 6144.0, 6148.0, 6149.0, 6150.0, 6153.0, 6157.0, 6165.0, 6168.0, 6169.0, 6185.0, 6190.0, 6203.0, 6205.0, 6214.0, 6216.0, 6218.0, 6220.0, 6225.0, 6227.0, 6230.0, 6239.0, 6252.0, 6263.0, 6278.0, 6288.0, 6289.0, 6297.0, 6299.0, 6302.0, 6306.0, 6316.0, 6325.0, 6333.0, 6335.0, 6348.0, 6355.0, 6364.0, 6365.0, 6369.0, 6375.0, 6383.0, 6392.0, 6397.0, 6400.0, 6401.0, 6406.0, 6411.0, 6417.0, 6422.0, 6428.0, 6430.0, 6437.0, 6438.0, 6442.0, 6450.0, 6452.0, 6458.0, 6460.0, 6470.0, 6481.0, 6483.0, 6486.0, 6493.0, 6505.0, 6513.0, 6532.0, 6565.0, 6578.0, 6609.0, 6617.0, 6621.0, 6626.0, 6642.0, 6651.0, 6680.0, 6980.0, 7280.0, 7580.000000000001, 7880.000000000001, 8180.000000000001, 8480.0, 8780.0, 9080